# Lab | Making predictions with logistic regression

In this lab, you will be using the [Sakila](https://dev.mysql.com/doc/sakila/en/) database of movie rentals.

In order to optimize our inventory, we would like to know which films will be rented next month and we are asked to create a model to predict it.


### Instructions

1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features. Use the data from 2005.
2. Create a query to get the list of films and a boolean indicating if it was rented last month (May 2005). This would be our target variable.
3. Read the data into a Pandas dataframe.
4. Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.
5. Create a logistic regression model to predict this variable from the cleaned data.
6. Evaluate the results.


## Import libraries and get database password

In [164]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input

password = getpass.getpass()

## Get database data through SQL query

In [276]:
# get the data
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = ''' 
            select title, rental_duration, rental_rate, length, replacement_cost, rating, category, rented_may
            from(
            select film_id,
            case
            when month(rental_date) = 5 then True
            else False
            end as rented_may
            from rental
            join inventory using(inventory_id)
            join film using(film_id)
            join film_category using(film_id)
            join category using (category_id)
            where year(rental_date) = 2005 and month(rental_date) = 5
            group by film_id) t1
            right join(
            select rental_date, title, film_id, rental_duration, rental_rate, length, replacement_cost, rating, category.name as category
            from rental
            join inventory using(inventory_id)
            join film using(film_id)
            join film_category using(film_id)
            join category using (category_id)
            where year(rental_date) = 2005
            group by film_id
            ) t2
            using (film_id)
            order by title asc'''

data = pd.read_sql_query(query, engine)
data.head(5)

,title,rental_duration,rental_rate,length,replacement_cost,rating,category,rented_may
0,ACADEMY DINOSAUR,6,0.99,86,20.99,PG,Documentary,1.0
1,ACE GOLDFINGER,3,4.99,48,12.99,G,Horror,NaN
2,ADAPTATION HOLES,7,2.99,50,18.99,NC-17,Documentary,1.0
3,AFFAIR PREJUDICE,5,2.99,117,26.99,G,Horror,1.0
4,AFRICAN EGG,6,2.99,130,22.99,G,Family,1.0


In [277]:
# Checking data types of columns
data.dtypes

title                object
rental_duration       int64
rental_rate         float64
length                int64
replacement_cost    float64
rating               object
category             object
rented_may          float64
dtype: object

In [278]:
# drop this column
data['replacement_cost'].value_counts()

20.99    55
21.99    55
22.99    54
29.99    52
12.99    52
27.99    51
13.99    50
14.99    48
11.99    47
17.99    46
10.99    46
26.99    45
19.99    45
23.99    44
25.99    41
9.99     40
28.99    40
18.99    40
24.99    37
16.99    36
15.99    34
Name: replacement_cost, dtype: int64

In [279]:
# Has to be categorized
data['rental_duration'].value_counts()

6    203
3    197
4    194
5    186
7    178
Name: rental_duration, dtype: int64

In [280]:
# Has to be categorized
data['rental_rate'].value_counts()

0.99    326
4.99    320
2.99    312
Name: rental_rate, dtype: int64

In [281]:
# Bin the movies length with the standard 0,90,120,150-max durations
data['length'].describe()

count    958.000000
mean     115.490605
std       40.471844
min       46.000000
25%       80.250000
50%      114.000000
75%      150.000000
max      185.000000
Name: length, dtype: float64

In [282]:
# Checking for undisired nans
data.isna().sum()

title                 0
rental_duration       0
rental_rate           0
length                0
replacement_cost      0
rating                0
category              0
rented_may          272
dtype: int64

The Nan's in rented_may actually represent that the movie wasn't rented in may, since this is a boolean field we are going to  fill it with 0 instead of dropping.

In [283]:
data['rented_may'] = data['rented_may'].fillna(0)

In [284]:
data.head()

,title,rental_duration,rental_rate,length,replacement_cost,rating,category,rented_may
0,ACADEMY DINOSAUR,6,0.99,86,20.99,PG,Documentary,1.0
1,ACE GOLDFINGER,3,4.99,48,12.99,G,Horror,0.0
2,ADAPTATION HOLES,7,2.99,50,18.99,NC-17,Documentary,1.0
3,AFFAIR PREJUDICE,5,2.99,117,26.99,G,Horror,1.0
4,AFRICAN EGG,6,2.99,130,22.99,G,Family,1.0


In [285]:
# Arranging columns to better understanding
data = data[['title','category', 'rating', 'length','rental_rate','replacement_cost','rental_duration','rented_may']]
data.head()

,title,category,rating,length,rental_rate,replacement_cost,rental_duration,rented_may
0,ACADEMY DINOSAUR,Documentary,PG,86,0.99,20.99,6,1.0
1,ACE GOLDFINGER,Horror,G,48,4.99,12.99,3,0.0
2,ADAPTATION HOLES,Documentary,NC-17,50,2.99,18.99,7,1.0
3,AFFAIR PREJUDICE,Horror,G,117,2.99,26.99,5,1.0
4,AFRICAN EGG,Family,G,130,2.99,22.99,6,1.0


### Summary of actions to take.


+ Columns to categorical encode: title, category, rating, rental_rate, rental duration.
+ Columns to drop: replacement_cost
+ Columns to bin: length

+ Target column: rented_may

In [286]:
# droping column replacement_cost
data = data.drop(['replacement_cost'], axis = 1)
data.head()

,title,category,rating,length,rental_rate,rental_duration,rented_may
0,ACADEMY DINOSAUR,Documentary,PG,86,0.99,6,1.0
1,ACE GOLDFINGER,Horror,G,48,4.99,3,0.0
2,ADAPTATION HOLES,Documentary,NC-17,50,2.99,7,1.0
3,AFFAIR PREJUDICE,Horror,G,117,2.99,5,1.0
4,AFRICAN EGG,Family,G,130,2.99,6,1.0


In [287]:
# Columns to categorical encode
data.dtypes

title               object
category            object
rating              object
length               int64
rental_rate        float64
rental_duration      int64
rented_may         float64
dtype: object

In [288]:
# convert rental_rate and rental_duration to object
data['rental_rate'] = data['rental_rate'].astype(object)
data['rental_duration'] = data['rental_duration'].astype(object)

In [290]:
# making bins for length 0,90,120,150-max
lbl = ['short', 'normal', 'long', 'extended'] 
data['length'] = pd.cut(data['length'],[0,90,120,150,int(data['length'].max())], labels=lbl) 
data['length'].value_counts()

short       311
extended    233
normal      207
long        207
Name: length, dtype: int64

In [291]:
# X-y Split
y = data['rented_may']
X = data.drop(['rented_may'], axis=1)
    
# Train test Splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [292]:
X_test.head()

,title,category,rating,length,rental_rate,rental_duration
836,SWEETHEARTS SUSPECTS,Children,G,normal,0.99,3
477,KRAMER CHOCOLATE,Sports,R,extended,2.99,3
350,GOSFORD DONNIE,Action,G,long,4.99,5
891,VACATION BOONDOCK,Sci-Fi,R,long,2.99,4
855,TITANS JERK,Sci-Fi,PG,normal,4.99,4


In [293]:
# This encoder only will be run one time and will be good for both test and train sets
onehotencoder = OneHotEncoder(drop='first', handle_unknown = 'ignore').fit(X_train)

# One hot encoder funtion
def encode_cats(X_cat):
    encoded = onehotencoder.transform(X_cat).toarray()
    cols = onehotencoder.get_feature_names(input_features=X_cat.columns)
    return pd.DataFrame(encoded, columns=cols)

X_train = encode_cats(X_train)
X_train.head()

C:\Users\VideoMachine\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,title_ACE GOLDFINGER,title_ADAPTATION HOLES,title_AFFAIR PREJUDICE,title_AFRICAN EGG,title_AGENT TRUMAN,title_AIRPLANE SIERRA,title_AIRPORT POLLOCK,title_ALABAMA DEVIL,title_ALADDIN CALENDAR,title_ALAMO VIDEOTAPE,...,rating_R,length_long,length_normal,length_short,rental_rate_2.99,rental_rate_4.99,rental_duration_4,rental_duration_5,rental_duration_6,rental_duration_7
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [294]:
X_test = encode_cats(X_test)
X_test.head()

C:\Users\VideoMachine\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\VideoMachine\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,title_ACE GOLDFINGER,title_ADAPTATION HOLES,title_AFFAIR PREJUDICE,title_AFRICAN EGG,title_AGENT TRUMAN,title_AIRPLANE SIERRA,title_AIRPORT POLLOCK,title_ALABAMA DEVIL,title_ALADDIN CALENDAR,title_ALAMO VIDEOTAPE,...,rating_R,length_long,length_normal,length_short,rental_rate_2.99,rental_rate_4.99,rental_duration_4,rental_duration_5,rental_duration_6,rental_duration_7
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


## Using Logistic regression model

In [295]:
from sklearn.linear_model import LogisticRegression

classification = LogisticRegression(solver='lbfgs').fit(X_train, y_train)
predictions_clf = classification.predict(X_test)

## Using Kneighbors Classifier

In [296]:
from sklearn import neighbors

classification_K = neighbors.KNeighborsClassifier(n_neighbors=4, weights='uniform')
classification_K.fit(X_train, y_train)
predictions_clf_K = classification_K.predict(X_test)

## Evaluation metrics

In [317]:
print('\033[1m'+'\033[91m'+'\n       Classification scores\n'+'\033[0m')
print('Logistic regression Model: ',round(classification.score(X_test, y_test)*100,2),'%')
print('Kneighbors classifier:     ',round(classification_K.score(X_test, y_test)*100,2),'%')


       Classification scores

Logistic regression Model:  70.83 %
Kneighbors classifier:      54.17 %


In [314]:
print('\033[1m'+'\033[91m'+'Test set quantity results:\n\n '+'\033[0m',pd.Series(y_test).value_counts())
print('\033[1m'+'\033[91m'+'\nLogistic regression Model:\n '+'\033[0m',pd.Series(predictions_clf).value_counts())
print('\033[1m'+'\033[91m'+'\nKneighbors classifier:    \n '+'\033[0m',pd.Series(predictions_clf_K).value_counts())

Test set quantity results:

  1.0    136
0.0     56
Name: rented_may, dtype: int64

Logistic regression Model:
  1.0    184
0.0      8
dtype: int64

Kneighbors classifier:    
  1.0    142
0.0     50
dtype: int64


### Confusion Matrix

In [312]:
print('\033[1m'+'\033[91m'+'      Logistic regression\n'+'\033[0m')
array = confusion_matrix(y_test, predictions_clf)
print('             Predicted Labels')
print('             |   A    |     B')    
print('             -------------------')
print('True label A | ',array[0][0],'    |   ', array[0][1])
print('             -------------------')
print('           B | ',array[1][0],'    |  ', array[1][1])

      Logistic regression

             Predicted Labels
             |   A    |     B
             -------------------
True label A |  4     |    52
             -------------------
           B |  4     |   132


In [313]:
print('\033[1m'+'\033[91m'+'      Kneighbors classifier\n'+'\033[0m')
array = confusion_matrix(y_test, predictions_clf_K)
print('             Predicted Labels')
print('             |   A    |     B')    
print('             -------------------')
print('True label A | ',array[0][0],'    |   ', array[0][1])
print('             -------------------')
print('           B | ',array[1][0],'   |   ', array[1][1])

      Kneighbors classifier

             Predicted Labels
             |   A    |     B
             -------------------
True label A |  9     |    47
             -------------------
           B |  41    |    95


### Summary

Looking at the scores the logistics regression is better no doubt with 70%, but if we look at the quantity of 0's and 1's looking at the expected results quantities and comparing them to both methods quantities of each results we get the idea that the Kneighbors results are almost on spot just missing 6 results, although keeping in mind that the order of the results matters because in this case it will be associated with a particular movie we can see that Kneighbors is pretty un-usable missing 88 results, while the Logistic regression "only" misses 56.

I can't say that any method is really usable but I can say that the logistics regression in this case is more reliable, but I wouldn't lean heavilly on this model to make any business decision, it is tough a good rough guide.

One thing that I think that this model shows us is that the very nature of renting videos changes with time with new releases and that people normally want to rent new movies.
